# **先修知识**

**转置卷积（Transposed Convolution），也称为反卷积（Deconvolution）或分数步长卷积（Fractionally Strided Convolution）**，是深度学习中的一种卷积操作，通常用于图像处理、语音处理和语义分割等任务。**它与标准卷积相反，不是从输入中提取特征，而是将特征映射放大到更大的空间。**

下面是转置卷积的一些关键概念和原理：

1. **卷积操作回顾**：在标准卷积操作中，卷积核在输入图像上滑动，对输入的局部区域执行加权和操作，生成输出特征图。这导致了从较大输入到较小输出的特征提取。

2. **转置卷积操作**：转置卷积与标准卷积相反。**它是一种从小尺寸输入到大尺寸输出的操作**。转置卷积的**核心思想**是将输入的特征图扩大，以恢复或生成更高分辨率的特征图。**它通过在输出上滑动卷积核，将值从输出传播回输入，并进行相应的加权和求和操作。**

3. **步长（Stride）和填充（Padding）**：转置卷积可以使用与标准卷积相同的步长和填充参数。步长控制输出的尺寸，填充可以用来控制输出尺寸和输入输出对齐。

4. **反卷积的数学表示**：转置卷积的数学表示通常使用矩阵乘法和卷积核的转置来实现。它可以看作是输入和卷积核之间的一种线性映射。

5. **应用领域**：**转置卷积常用于语义分割中，将低分辨率的特征图恢复到与输入图像相同的分辨率，以生成像素级别的标签预测**。它还在生成对抗网络（GANs）中用于从噪声生成高分辨率图像，以及在图像分析和处理任务中用于上采样。

需要注意的是，"转置卷积" 这个术语有时可能会导致混淆，**因为它并不真正执行卷积的反操作，而是一种上采样技术**。在某些文献中，人们也可能称之为 "反卷积"，但这并不是传统卷积的逆运算。

总之，转置卷积是一种用于特征图上采样的重要工具，通常在深度学习中用于处理高分辨率特征图的生成和恢复任务。它允许将低分辨率特征映射扩展到更大的空间，以便进行各种图像和信号处理任务。

> **恢复卷积前的图像尺寸，而不是恢复原始值。**

---------------------------------

# 转置卷积
- 卷积不会增大输入的高宽，通常要么不变、要么减半
- 转置卷积则可以用来增大输入高宽

![卷积核为 $2\times 2$ 的转置卷积。阴影部分是中间张量的一部分，也是用于计算的输入和卷积核张量元素。 ](img/trans_conv.svg)

$$Y[i:i+h, j:j+w] += X[i, j] * K$$

## **为什么称之为“转置”**
- 对于卷积 $Y = X \star W$
    - 可以对 W 构造一个 V，使得卷积等价于矩阵乘法 $Y^` = V X`$
    - 这里的 $X^`$ 和 $Y^`$ 是 X 和 Y 的展开向量
- 转置卷积则等价于 $Y^` = V^T X^`$
- 如果卷积将输入从(h,w)变成了(h`,w`)
    - 同样超参数的转置卷积则从(h`,w`)变成(h,w)

----------------------------------
到目前为止，我们所见到的卷积神经网络层，例如卷积层（ `sec_conv_layer` ）和汇聚层（ `sec_pooling` ），通常会减少下采样输入图像的空间维度（高和宽）。
**然而如果输入和输出图像的空间维度相同，在以像素级分类的语义分割中将会很方便。**
例如，输出像素所处的通道维可以保有输入像素在同一位置上的分类结果。

**为了实现这一点，尤其是在空间维度被卷积神经网络层缩小后，我们可以使用另一种类型的卷积神经网络层，它可以增加上采样中间层特征图的空间维度。**
本节将介绍
**转置卷积**（transposed convolution） :cite:`Dumoulin.Visin.2016`，
用于逆转下采样导致的空间尺寸减小。


In [1]:
import torch
from torch import nn
from d2l import torch as d2l

## 基本操作

让我们暂时忽略通道，从基本的转置卷积开始，设步幅为1且没有填充。
假设我们有一个$n_h \times n_w$的输入张量和一个$k_h \times k_w$的卷积核。
以步幅为1滑动卷积核窗口，**每行$n_w$次，每列$n_h$次，共产生$n_h n_w$个中间结果。**
**每个中间结果都是一个$(n_h + k_h - 1) \times (n_w + k_w - 1)$的张量，初始化为0。**
为了计算每个中间张量，输入张量中的每个元素都要乘以卷积核，从而使所得的$k_h \times k_w$张量替换中间张量的一部分。
请注意，每个中间张量被替换部分的位置与输入张量中元素的位置相对应。
最后，所有中间结果相加以获得最终结果。

例如， `fig_trans_conv` 解释了如何为$2\times 2$的输入张量计算卷积核为$2\times 2$的转置卷积。

![卷积核为 $2\times 2$ 的转置卷积。阴影部分是中间张量的一部分，也是用于计算的输入和卷积核张量元素。 ](img/trans_conv.svg)

我们可以对输入矩阵`X`和卷积核矩阵`K`(**实现基本的转置卷积运算**)`trans_conv`。


In [3]:
def trans_conv(X, K):
    # 获取卷积核的高度和宽度
    h, w = K.shape
    
    # 计算输出特征图的大小，通过在输入的高度和宽度上添加卷积核大小减一的边界
    Y = torch.zeros((X.shape[0] + h - 1, X.shape[1] + w - 1))
    
    # 遍历输入特征图的每个元素
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            # 在输出特征图上相应的位置上添加卷积结果
            Y[i: i + h, j: j + w] += X[i, j] * K
            
    return Y

**与通过卷积核“减少”输入元素的常规卷积（在 `sec_conv_layer` 中）相比，转置卷积通过卷积核“广播”输入元素，从而产生大于输入的输出。**
我们可以通过 `fig_trans_conv` 来构建输入张量`X`和卷积核张量`K`从而[**验证上述实现输出**]。
此实现是基本的二维转置卷积运算。


In [10]:
X = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
trans_conv(X, K)

tensor([[ 0.,  0.,  1.],
        [ 0.,  4.,  6.],
        [ 4., 12.,  9.]])

或者，当输入`X`和卷积核`K`都是四维张量时，我们可以[**使用高级API获得相同的结果**]。


In [14]:
# 将输入特征图 X 和卷积核 K 重新调整形状为 (batch_size=1, channels=1, height=2, width=2)
X, K = X.reshape(1, 1, 2, 2), K.reshape(1, 1, 2, 2)

# 创建一个转置卷积层 tconv，具有 1 个输入通道，1 个输出通道，卷积核大小为 (kernel_size=2)，且没有偏置
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, bias=False)

# 将卷积核 K 的数据设置为 tconv 的权重数据
tconv.weight.data = K

# 使用转置卷积层 tconv 对输入特征图 X 进行转置卷积操作
tconv(X)

tensor([[[[ 0.,  0.,  1.],
          [ 0.,  4.,  6.],
          [ 4., 12.,  9.]]]], grad_fn=<ConvolutionBackward0>)

## [**填充、步幅和多通道**]

**与常规卷积不同，在转置卷积中，填充被应用于的输出（常规卷积将填充应用于输入）。**
例如，当将高和宽两侧的填充数指定为1时，转置卷积的输出中将删除第一和最后的行与列。


In [15]:
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, padding=1, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[4.]]]], grad_fn=<ConvolutionBackward0>)

**在转置卷积中，步幅被指定为中间结果（输出），而不是输入。**
使用 `fig_trans_conv` 中相同输入和卷积核张量，将步幅从1更改为2会增加中间张量的高和权重，因此输出张量在 `fig_trans_conv_stride2` 中。

![卷积核为$2\times 2$，步幅为2的转置卷积。阴影部分是中间张量的一部分，也是用于计算的输入和卷积核张量元素。](img/trans_conv_stride2.svg)

以下代码可以验证 `fig_trans_conv_stride2` 中步幅为2的转置卷积的输出。


In [16]:
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, stride=2, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[0., 0., 0., 1.],
          [0., 0., 2., 3.],
          [0., 2., 0., 3.],
          [4., 6., 6., 9.]]]], grad_fn=<ConvolutionBackward0>)

**对于多个输入和输出通道**，转置卷积与常规卷积以相同方式运作。
假设输入有$c_i$个通道，且转置卷积为每个输入通道分配了一个$k_h\times k_w$的卷积核张量。
**当指定多个输出通道时，每个输出通道将有一个$c_i\times k_h\times k_w$的卷积核。**

同样，如果我们将$\mathsf{X}$代入卷积层$f$来输出$\mathsf{Y}=f(\mathsf{X})$，**并创建一个与$f$具有相同的超参数、但输出通道数量是$\mathsf{X}$中通道数的转置卷积层$g$，那么$g(Y)$的形状将与$\mathsf{X}$相同。**
下面的示例可以解释这一点。


In [17]:
X = torch.rand(size=(1, 10, 16, 16))
conv = nn.Conv2d(10, 20, kernel_size=5, padding=2, stride=3)
tconv = nn.ConvTranspose2d(20, 10, kernel_size=5, padding=2, stride=3)
tconv(conv(X)).shape == X.shape

True

## [**与矩阵变换的联系**]

**转置卷积为何以矩阵变换命名呢？**
让我们首先看看如何**使用矩阵乘法**来实现卷积。
在下面的示例中，我们定义了一个$3\times 3$的输入`X`和$2\times 2$卷积核`K`，然后使用`corr2d`函数计算卷积输出`Y`。


In [18]:
X = torch.arange(9.0).reshape(3, 3)
K = torch.tensor([[1.0, 2.0], [3.0, 4.0]])
Y = d2l.corr2d(X, K)
Y

tensor([[27., 37.],
        [57., 67.]])

接下来，我们将卷积核`K`重写为包含大量0的稀疏权重矩阵`W`。
权重矩阵的形状是（$4$，$9$），其中非0元素来自卷积核`K`。


In [19]:
# 定义一个函数 kernel2matrix，将输入的卷积核 K 转换为一个权重矩阵 W
def kernel2matrix(K):
    # 创建一个包含 5 个元素的零张量 k
    k = torch.zeros(5)
    
    # 创建一个包含 4 行 9 列的零矩阵 W
    W = torch.zeros((4, 9))
    
    # 将卷积核 K 的部分元素复制到张量 k 中
    # 这里选择了卷积核 K 的不同部分来填充张量 k，具体方式如下：
    # - k 的前两个元素（k[:2]）被填充为卷积核 K 的第一行元素（K[0, :]）
    # - k 的中间元素（k[2]）被留空
    # - k 的后两个元素（k[3:5]）被填充为卷积核 K 的第二行元素（K[1, :]）
    k[:2], k[3:5] = K[0, :], K[1, :]
    
    # 将张量 k 的值分别填充到矩阵 W 的不同行中
    # 具体方式如下：
    # - W 的第一行（W[0, :5]）被填充为张量 k 的前五个元素（k）
    # - W 的第二行（W[1, 1:6]）被填充为张量 k 的第二个到第六个元素（k[1:]）
    # - W 的第三行（W[2, 3:8]）被填充为张量 k 的第四个到第八个元素（k[3:]）
    # - W 的第四行（W[3, 4:]）被填充为张量 k 的最后两个元素（k[4:]）
    W[0, :5], W[1, 1:6], W[2, 3:8], W[3, 4:] = k, k, k, k
    
    # 返回生成的权重矩阵 W
    return W

# 调用 kernel2matrix 函数，将卷积核 K 转换为权重矩阵 W
W = kernel2matrix(K)

# 输出生成的权重矩阵 W
W

tensor([[1., 2., 0., 3., 4., 0., 0., 0., 0.],
        [0., 1., 2., 0., 3., 4., 0., 0., 0.],
        [0., 0., 0., 1., 2., 0., 3., 4., 0.],
        [0., 0., 0., 0., 1., 2., 0., 3., 4.]])

逐行连结输入`X`，获得了一个长度为9的矢量。
然后，`W`的矩阵乘法和向量化的`X`给出了一个长度为4的向量。
重塑它之后，可以获得与上面的原始卷积操作所得相同的结果`Y`：我们刚刚使用矩阵乘法实现了卷积。


In [10]:
Y == torch.matmul(W, X.reshape(-1)).reshape(2, 2)

tensor([[True, True],
        [True, True]])

同样，我们可以使用矩阵乘法来实现转置卷积。
在下面的示例中，我们将上面的常规卷积$2 \times 2$的输出`Y`作为转置卷积的输入。
想要通过矩阵相乘来实现它，我们只需要将权重矩阵`W`的形状转置为$(9, 4)$。


In [11]:
Z = trans_conv(Y, K)
Z == torch.matmul(W.T, Y.reshape(-1)).reshape(3, 3)

tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])

抽象来看，给定输入向量$\mathbf{x}$和权重矩阵$\mathbf{W}$，卷积的前向传播函数可以通过将其输入与权重矩阵相乘并输出向量$\mathbf{y}=\mathbf{W}\mathbf{x}$来实现。
由于反向传播遵循链式法则和$\nabla_{\mathbf{x}}\mathbf{y}=\mathbf{W}^\top$，卷积的反向传播函数可以通过将其输入与转置的权重矩阵$\mathbf{W}^\top$相乘来实现。
因此，转置卷积层能够交换卷积层的正向传播函数和反向传播函数：它的正向传播和反向传播函数将输入向量分别与$\mathbf{W}^\top$和$\mathbf{W}$相乘。


## 小结

* 与通过卷积核减少输入元素的常规卷积相反，转置卷积通过卷积核广播输入元素，从而产生形状大于输入的输出。
* 如果我们将$\mathsf{X}$输入卷积层$f$来获得输出$\mathsf{Y}=f(\mathsf{X})$并创造一个与$f$有相同的超参数、但输出通道数是$\mathsf{X}$中通道数的转置卷积层$g$，那么$g(Y)$的形状将与$\mathsf{X}$相同。
* 我们可以使用矩阵乘法来实现卷积。转置卷积层能够交换卷积层的正向传播函数和反向传播函数。


## 练习

1. 在 `subsec-connection-to-mat-transposition` 中，卷积输入`X`和转置的卷积输出`Z`具有相同的形状。他们的数值也相同吗？为什么？
1. 使用矩阵乘法来实现卷积是否有效率？为什么？